In [1]:
import os
import numpy as np
import torch
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import operator
import io
import time
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data.sampler import SubsetRandomSampler
import pandas as pd
import scipy.signal as sg
from pathlib import Path
import cv2
from google.colab.patches import cv2_imshow
from PIL import Image
from matplotlib import pyplot as plt 
import PIL.Image
import numpy as np
import cv2
import cv2 as cv
import csv
from google.colab.patches import cv2_imshow

In [2]:
!sudo pip install pytesseract
!sudo apt install tesseract-ocr
!sudo apt install libtesseract-dev
!pip install requests fuzzywuzzy pandas pyjstat numpy plotly matplotlib seaborn geopy google folium pandas googlemaps

  Created wheel for pytesseract: filename=pytesseract-0.3.8-py2.py3-none-any.whl size=14072 sha256=88f6a36036360650a6281c5399a940e7673a79689572cbcfcaa683739bb8cd84
  Stored in directory: /root/.cache/pip/wheels/a4/89/b9/3f11250225d0f90e5454fcc30fd1b7208db226850715aa9ace
Successfully built pytesseract
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 37 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:3 http:/

In [3]:
from pytesseract import image_to_string
import pytesseract
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

/usr/local/lib/python3.7/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
%cd /content/drive/MyDrive/Github/

/content/drive/MyDrive/Github


In [7]:
#one time run to initialize github repo
!git init flyer_ai_model
%cd flyer_ai_model/
%ls -a

Initialized empty Git repository in /content/drive/My Drive/Github/flyer_ai_model/.git/
/content/drive/My Drive/Github/flyer_ai_model
.git/


In [9]:
!git add .

In [10]:
!git status

On branch master

No commits yet

nothing to commit (create/copy files and use "git add" to track)


In [ ]:
class ImageFolderWithFileName(datasets.ImageFolder):
    """Custom dataset that includes image file paths. Extends
    torchvision.datasets.ImageFolder
    """

    # override the __getitem__ method. this is the method that dataloader calls
    def __getitem__(self, index):
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithFileName, self).__getitem__(index)
        # the image file path
        path = Path(self.imgs[index][0]).name
        if path.endswith("jpg") or path.endswith("png"):
          if "_" not in path:
            path = path.split(".")[0]
        #print('kan', type(original_tuple))
        ret_with_name = (original_tuple[:1] + (path,))
        return ret_with_name

data_dir = "/content/drive/My Drive/Github/flyer_images/"
test_data = ImageFolderWithFileName(data_dir) # our custom dataset

# **Sample Image**

In [ ]:
test_data[0][0]

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
test_data[0][1]

'week_10_page_1.jpg'

# Main

In [ ]:
def textSectionChecker(croppedImage):

    #testing
    #hsv = cv.cvtColor(croppedImage, cv.COLOR_BGR2HSV)
    #lower_black = np.array([0,0,0])
    #upper_black = np.array([2,2,2])
    #mask = cv2.inRange(hsv, lower_black, upper_black)

    #res = cv2.bitwise_and(croppedImage,croppedImage, mask=mask)
    imgBlack = croppedImage.copy()
    imgBlack = cv.cvtColor(imgBlack, cv.COLOR_BGR2GRAY)
    imgBlack[imgBlack > 40] = 255
    cv2_imshow(imgBlack)

    print('llllllllllll')
    #works

    imgray = cv.cvtColor(croppedImage, cv.COLOR_BGR2GRAY)
    ret,thresh1 = cv2.threshold(imgray, 0, 255,cv2.THRESH_OTSU|cv2.THRESH_BINARY_INV)
    rect_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (15, 3))
    dilation = cv2.dilate(thresh1, rect_kernel, iterations = 1)
    kernel = np.ones((1,1),np.uint8)
    dilation2 = cv2.dilate(dilation,kernel,iterations = 1)
    cv2_imshow(dilation2)
    contours, hierarchy = cv2.findContours(dilation2, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    im2 = croppedImage.copy()
    textContours = 0
    print('-------------------------------')
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        cv2.rectangle(im2, (x, y), (x + w, y + h), (0, 255, 0), 1)
        element = im2[y:y+h,x:x+w]
        currentImage = Image.fromarray(element)
        output = pytesseract.image_to_string(currentImage)
        #if empty
        if output is None:
          continue
        print(output)
        textContours + 1
    cv2_imshow(im2)
    print('-------------------------------')
    if textContours > 4:
      return 1
    return 0

In [ ]:
########this is for product
product = []
row = 0
with open("/content/drive/My Drive/Github/product_dictionary.csv") as csvfile:
  reader = csv.reader(csvfile, delimiter= ',')
  for row in reader:
    if row == 0:
      row = 1
      continue
    product.append(row)

#####this is for unit
unit = []
row = 0
with open("/content/drive/My Drive/Github/units_dictionary.csv") as csvfile:
  reader = csv.reader(csvfile, delimiter= ',')
  for row in reader:
    if row == 0:
      row = 1
      continue
    unit.append(row[0])

In [ ]:
def convertStringToFloat(element):
  nu = 0
  try:
    nu = float(element)
  except ValueError:
    pass
    #print("Not a float")
  return(nu)

def convertStringToInt(element):
  nu = 0
  try:
    nu = int(element)
  except ValueError:
    pass
    #print("Not an int")
  return(nu)

#this function returns the data for each list element based off different conditions
def analyzeData(name, output):

  #status = ["set", "unset", "multiplier"]

  status = {
      "unit_promo_price": "unset",
      "uom": "unset",
      "least_unit_for_promo": "unset",
      "save_per_unit": "unset",
      "discount": "unset"
  }

  unit_promo_price = ''
  uom = ''
  least_unit_for_promo = 1 # default
  save_per_unit = ''
  discount = ''
  organic = 0 #default not organic

  for line in output.splitlines():
    #print('--------')
    #print(line)
    
    #if organic ever appears, then organic = 1
    if("organic" in line.lower()):
      organic = 1

    list_of_words = (line.lower()).split()

    for item in list_of_words:

      if (item.lower()).replace(".", "") in unit:
        uom = (item.lower()).replace(".", "")
        #print("unit found", uom)
        status["uom"] = "set"

    #if any(i in str(line.lower()) for i in unit):
    #t = set(unit).intersection(list_of_words)
    #if t is not None:
    #  print('common unit', t)

    #print(line.lower())
    #print(list_of_words)
    if("off" in list_of_words and status["discount"] == "unset"):
      word_before = list_of_words[list_of_words.index("off") - 1]
      if "half" == word_before:
        discount = 0.5
        status["discount"] = "set"
      elif "%" in word_before:
        percent = word_before[0:word_before.index("%")]
        if convertStringToInt(percent) != 0:
          discount = convertStringToInt(percent)/100
          status["discount"] = "set"
        else:
          print("unknown percentage")

    #Case 1: save amount
    #print(line.lower())
    #print(list_of_words)
    if("save" in list_of_words):
      if(len(list_of_words) <= list_of_words.index("save") + 1):
        continue;

      next_word = list_of_words[list_of_words.index("save") + 1]
      #print("save:", next_word)

      if "/" in next_word:
        save_per_unit = next_word[0:next_word.index("/")]
        if "$" in save_per_unit:
          save_per_unit = convertStringToFloat(save_per_unit[(save_per_unit.index("$") + 1):])
        elif "¢" in save_per_unit:
          save_per_unit = convertStringToFloat(save_per_unit[:save_per_unit.index("¢")])/100
        #print("float", save_per_unit)
        uom_inner = next_word[(next_word.index("/") + 1):]
        if uom_inner.lower() in ["ib", "lb", "ib.", "lb."]:
          uom = 'lb'
        #if unit_inner.lower() in unit :
        else:
          uom = uom_inner.lower()
        #print("unit", uom)
        #else:
        #  print("unknown unit promo price")
        status['save_per_unit'] = "set"
        status["uom"] = "set"

      #save $ on unit
      elif "on" in list_of_words:
        before_on = list_of_words[list_of_words.index("on") - 1]

        if "$" in before_on:
          save_per_unit = convertStringToFloat(before_on[(before_on.index("$") + 1):])
          status['save_per_unit'] = "set"
          #print("float", save_per_unit)
        elif "¢" in before_on:
          save_per_unit = convertStringToFloat(before_on[:before_on.index("¢")])/100
          status['save_per_unit'] = "set"
          #print("cent", save_per_unit)
        else:
          #print("unknown continue")
          continue

        if(len(list_of_words)> (list_of_words.index("on") + 1)):
          after_on = list_of_words[list_of_words.index("on") + 1]
        else:
          continue

        if convertStringToInt(after_on) != 0:
          least_unit_for_promo = convertStringToInt(after_on)
          save_per_unit = save_per_unit/least_unit_for_promo
          status['least_unit_for_promo'] = "set"
          #print("promo unit", least_unit_for_promo)

      elif "$" in next_word:
        save_per_unit = convertStringToFloat(next_word[(next_word.index("$") + 1):])
        status['save_per_unit'] = "set"
        #print("float", save_per_unit)
      elif "¢" in next_word:
        save_per_unit = convertStringToInt(next_word[:next_word.index("¢")])/100
        status['save_per_unit'] = "set"
        #print("cent", save_per_unit)
      else:
        pass
        #print("unknown save")
    
    #CASE 2: not save and price is shown in $
    elif(len(list_of_words) == 1 and "$" in line.lower()):
      price = list_of_words[0]
      stringpricelen = len(price[(price.index("$")+1):])
      #print('99 test', price[(len(price)-3):])
      #assume divide by 1000
      if (stringpricelen > 2 and price[(len(price)-3):] == '99'):
        unit_promo_price = convertStringToFloat(price[(price.index("$")+1):])/1000
      else:
        unit_promo_price = convertStringToFloat(price[(price.index("$")+1):])/100
      status['unit_promo_price'] = "set"
      #print("cent", unit_promo_price)
    #CASE 3: not save and price is shown in ¢
    elif(len(list_of_words) == 1 and "¢" in line.lower()):
      price = list_of_words[0]
      unit_promo_price = convertStringToInt(price[:price.index("¢")])/100
      status['unit_promo_price'] = "set"
      #print("cent", unit_promo_price)

  #at the end do some calculations if possible before returning

  return unit_promo_price, uom, least_unit_for_promo, save_per_unit, discount, organic

In [ ]:
import re

In [ ]:
flyer_name = test_data[0][1]
print(flyer_name)

week_10_page_1.jpg


In [ ]:
flyer_name = test_data[0][1]
productDict = []

for i in range(len(test_data)):
    flyer_name = test_data[i][1]
    temp = flyer_name.split(".")
    print(temp)
    flyer_name = temp[0]

    #contour each flyer to find sections
    image = np.array(test_data[i][0])

    imgray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    ret,thresh1 = cv2.threshold(imgray, 0, 255,cv2.THRESH_OTSU|cv2.THRESH_BINARY_INV)
    rect_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (15, 3))
    dilation = cv2.dilate(thresh1, rect_kernel, iterations = 1)

    #dilate more
    kernel = np.ones((20,20),np.uint8)
    dilation2 = cv2.dilate(dilation,kernel,iterations = 1)

    #find contour for each section
    contours, hierarchy = cv2.findContours(dilation2, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    im2 = image.copy()
    for cnt in contours:
          x, y, w, h = cv2.boundingRect(cnt)
          cv2.rectangle(im2, (x, y), (x + w, y + h), (0, 255, 0), 1)
    #cv2_imshow(im2)

    #save cropped images for flyer
    cropped_imgs = {}
    #cropped_imgs = []
    for cnt in contours:
        im3 = image.copy()
        x, y, w, h = cv2.boundingRect(cnt)
        cropped = im3[y:y+h,x:x+w]
        #for overlap, save location in original image
        name = str(x) + "-" + str(x+w) + "-" + str(y) + "-" + str(y+h)
        #print(name)
        cropped_imgs[name] = cropped
        #cropped_imgs.append(cropped)

    #for key, entry in cropped_imgs.items():
    #    cv2_imshow(entry)

        
###############################################################################

    i = 0

    for key, element in cropped_imgs.items():
        #print("Iteration", i)
        #####get the main product name contour text
        imgOther = element.copy()
        imgBlack = element.copy()
        imgBlack = cv.cvtColor(imgBlack, cv.COLOR_BGR2GRAY)
        imgBlack[imgBlack > 40] = 255

        ret,thresh1 = cv2.threshold(imgBlack, 0, 255,cv2.THRESH_OTSU|cv2.THRESH_BINARY_INV)
        #cv2_imshow(thresh1)

        kernel = np.ones((6,6), np.uint8) 
        img_erosion = cv2.erode(thresh1, kernel, iterations=1) 
        #cv2_imshow(img_erosion) 

        kernel = np.ones((55,55), np.uint8) 
        img_dilation = cv2.dilate(img_erosion, kernel, iterations=1) 
        #cv2_imshow(img_dilation)

        #---Finding contours ---
        contours, hierarchy = cv2.findContours(img_dilation, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

        im3 = imgBlack.copy()
        name = ''
        for cnt in contours:
                x, y, w, h = cv2.boundingRect(cnt)
                cv2.rectangle(im3, (x, y), (x + w + 8, y + h), (0, 255, 0), 1)
                element = im3[y:y+h,x:x+w]
                currentImage = Image.fromarray(element)
                output = pytesseract.image_to_string(currentImage)
                #print('product name =', output)
                name = name + '^' + output
        name = name.replace("^^", "")
        name = name.replace("^", " ")
        name = name.replace("^", "")
        name = name.replace("\n", " ")
        name = name.replace("~", "")
        name = name.rstrip()
        name = name.lstrip()
        current = name.split("  ")
        name = current[0]
        i = i + 1
        if (name == '' or name == ' '):
          continue

        value = process.extract(name, product, limit=1, scorer=fuzz.token_sort_ratio)
        name = value[0][0][0]
        #print('product name:', name)
        #cv2_imshow(im3)
        
        ####get the rest of the text
        #boolTextSection = textSectionChecker(element)
        currentImage = Image.fromarray(imgOther)
        currentImage = currentImage.convert("L")
        output = pytesseract.image_to_string(currentImage)
        #print(output)
        #print("\n")
        #cv2_imshow(imgOther)

        unit_promo_price, uom, least_unit_for_promo, save_per_unit, discount, organic = analyzeData(name, output)

        list_element = {'flyer_name': flyer_name,
                      'product_name': name,
                      'unit_promo_price': unit_promo_price,
                      'uom': uom,
                      'least_unit_for_promo': least_unit_for_promo,
                      'save_per_unit': save_per_unit,
                      'discount': discount,
                      'organic': organic}

        productDict.append(list_element)  
        #print(list_element)

csv_columns = ['flyer_name', 'product_name', 'unit_promo_price', 'uom', 'least_unit_for_promo', 'save_per_unit', 'discount', 'organic']

with open("/content/drive/My Drive/Github/results.csv", 'w') as csvfile:
  writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
  writer.writeheader()
  for data in productDict:
    writer.writerow(data)

['week_10_page_1', 'jpg']
['week_10_page_2', 'jpg']
['week_10_page_3', 'jpg']
['week_10_page_4', 'jpg']
['week_11_page_1', 'jpg']
['week_11_page_2', 'jpg']
['week_11_page_3', 'jpg']
['week_11_page_4', 'jpg']


['week_12_page_1', 'jpg']
['week_12_page_2', 'jpg']
['week_12_page_3', 'jpg']
['week_12_page_4', 'jpg']


['week_13_page_1', 'jpg']


['week_13_page_2', 'jpg']
['week_13_page_3', 'jpg']
['week_13_page_4', 'jpg']
['week_14_page_1', 'jpg']
['week_14_page_2', 'jpg']
['week_14_page_3', 'jpg']
['week_14_page_4', 'jpg']
['week_15_page_1', 'jpg']
['week_15_page_2', 'jpg']
['week_15_page_3', 'jpg']
['week_15_page_4', 'jpg']
['week_16_page_1', 'jpg']
['week_16_page_2', 'jpg']
['week_16_page_3', 'jpg']
['week_16_page_4', 'jpg']
['week_17_page_1', 'jpg']
['week_17_page_2', 'jpg']
['week_17_page_3', 'jpg']


['week_17_page_4', 'jpg']
['week_18_page_1', 'jpg']
['week_18_page_2', 'jpg']
['week_18_page_3', 'jpg']
